In [1]:
from alpyne.client.alpyne_client import AlpyneClient
import numpy as np
import random
import time
from IPython.display import clear_output

In [2]:
sys.path.append("../..")
from thesis.q_learning.q_table import QTable
        

In [3]:
client = AlpyneClient("../../models\AGV_SlipCourse Exported\AGV_SlipCourse.zip", blocking = True, port = 51151)

C:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_70899_wm862gh9)
  warn(f"Unzipping to temporary directory ({tmp_dir})")


In [4]:
config = client.configuration_template
config.inSlipChance = 0.3
run = client.create_reinforcement_learning(config)

In [5]:
run = run.run()

In [6]:
num_episodes = 1000
max_steps_per_episode = 100

learning_rate = 0.1     #Alpha
discount_rate = 0.99    #Gamma

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001
n_actions = 4

In [7]:
q_table = QTable(n_actions)

In [8]:
rewards_all_episodes = []

In [10]:
# Q-learning algorithm
for episode in range(num_episodes):
    if episode % 10 == 1:
        print(episode)
    run.reset()
    run.wait_for_completion()
    state, _ = run.get_observation().values()
    
    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        action = client.action_template
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action.inAction = np.argmax(q_table[state,:])
        else:
            action.inAction = random.randint(0,n_actions-1)
            
        run.take_action(action)
        run.wait_for_completion()
        done = run.is_terminal()
        new_state, reward = run.get_observation().values()
        
        # Update Q-table
        q_table[state, action.inAction] = q_table[state, action.inAction] * (1-learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break        

    # Exploration rate decay   
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
        
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)
    
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), len(rewards_all_episodes)/1000)
count = 1000
print("*******Average reward per thousand episodes*******\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000
    
print("\n\n*******Q-table*******\n")
print(q_table)

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881
891
901
911
921
931
941
951
961
971
981
991
*******Average reward per thousand episodes*******

1000 :  -0.7420000000000005


*******Q-table*******

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# Old

In [7]:
obs_hist = []

In [8]:
obs = run.get_observation()
obs_hist.append(obs.values())
obs_hist

[[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 0]]

In [11]:
action = client.action_template
action.inAction = 1 #0-forward, 1-back, 2-wait, 3-toggle
run.take_action(action)

ModelError: ModelError[path=http://127.0.0.1:51151/runs/1/rl, status=400, error=Bad Request, msg=('{"Exception message":"Platform not in correct stage: (FINISH, '
 'true)","Platform '
 'state":{"message":"{\\"reason\\":\\"MODEL_METHOD\\",\\"next_event_time\\":125.0,\\"episode_count\\":1,\\"step_count\\":126,\\"model_time\\":124.0}","status":"COMPLETED"}}')]

In [87]:
run.reset()

In [88]:
run.is_terminal()

False

In [89]:
run.get_state()

(<RunStatus.PAUSED: 'PAUSED'>,
 {'reason': None,
  'next_event_time': 1.0,
  'episode_count': 2,
  'step_count': 1,
  'model_time': 0.0})